In [5]:
# 현재 노트북의 위치 확인
import os
from pathlib import Path
CURRENT_DIR = Path(os.getcwd())
print(CURRENT_DIR)

/Users/sungwook/Desktop/web/ag-isbnapi-fastapi/jupyter_research


In [15]:
# env 파일에서 ttb key 정보 가지고 오기
from dotenv import dotenv_values
config = dotenv_values(CURRENT_DIR.parent / ".env")
ALADIN_TTB_KEY=config["ALADIN_TTB_KEY"]

In [16]:
# 아래 코드에서 사용될 API 요청주소와 필요한 TTB 값
url = "http://www.aladin.co.kr/ttb/api/ItemLookUp.aspx"
ttbkey = ALADIN_TTB_KEY

In [17]:
# 서재정보
# 모던 자바스크립트 Deep Dive
book_example1 = {
    "isbn_type": "ISBN13",
    "isbn_str": "9791158392239",
}
# 2021 기분파 지게차 운전기능사 필기
book_example2 = {
    "isbn_type": "ISBN",
    "isbn_str": "k622633024",
}

In [19]:
import requests
from typing import Dict


def get_response(isbn_type, isbn_str):
    """isbn_type과 isbn_str이 담긴 dict를 받아 api response를 리턴"""
    res = requests.post(
        url,
        data = {
            "TTBKey": ttbkey,
            "itemIdType": isbn_type,
            "ItemId": isbn_str,
            "Cover": "Big",
            "output": "js"
        },
    )

    return res

In [70]:
# 정상응답시 200 리턴
response_example1 = get_response(**book_example1)
print(response_example1)

<Response [200]>


In [22]:
response_example1.status_code

200

In [89]:
response_example1.headers

{'Cache-Control': 'private', 'Content-Type': 'text/plain; charset=utf-8', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'Server': 'Microsoft-IIS/7.5', 'Set-Cookie': 'AladdinUS=ZKrK%2boUzjBg%3d&USA=0; domain=aladin.co.kr; expires=Tue, 29-Oct-2041 15:53:44 GMT; path=/', 'X-Powered-By': 'ASP.NET', 'P3P': "CP='CAO PSA CONi OTR OUR DEM ONL'", 'Date': 'Fri, 29 Oct 2021 15:53:44 GMT', 'Content-Length': '11800'}

In [39]:
f = open(CURRENT_DIR / "text.txt", "w")
f.write(response_example1.text)
f.close()
# response_example#.json() 은 사용불가. text 마지막에 세미콜론이 붙어있다.

In [65]:
import json

# response_example1.text 는 json을 str화 한 것이다.
# 해당 str 마지막에는 왜인지 모르지만 맨 마지막에 세미콜론이 들어가있다.
# 또한 '\t'와 같은 문자가 있을 경우 strict=False가 필요하다.
# str->json 은 load 과정이다.
# 반대 dump 과정에서 ensure_ascii=False가 필요하다.
# 저장된 json은 format documents 해서 볼 것.
json_example1 = json.loads(response_example1.text[:-1], strict=False)
j = open("data.json", "w")
json.dump(json_example1, j, ensure_ascii=False)
j.close()

In [85]:
def get_bookinfo(isbn_type, isbn_str) -> Dict:
    """서재정보를 받아 검색한뒤, api정보에서 쓸만한 정보들을 추출하여 dict로 반환하는 함수"""
    res = get_response(isbn_type, isbn_str)
    if res.status_code != 200:
        print("Adadin API Error: status code is not 200")
        return {"errors": "response status code is not 200"}
    json_res: Dict = json.loads(res.text[:-1], strict=False)
    useful_keyword = ["title", "description", "isbn13", "cover", "publisher", "priceSales", "pubDate", "author"]
    useful_data = {}
    for keyword in useful_keyword:
        useful_data[keyword] = json_res["item"][0][keyword]
    return useful_data

In [86]:
extract_example1 = get_bookinfo(**book_example1)

In [87]:
print(extract_example1)

{'title': '모던 자바스크립트 Deep Dive - 자바스크립트의 기본 개념과 동작 원리', 'description': '자바스크립트를 둘러싼 기본 개념을 정확하고 구체적으로 설명하고, 자바스크립트 코드의 동작 원리를 집요하게 파헤친다. 작성한 코드가 컴퓨터 내부에서 어떻게 동작할 것인지 예측하고, 명확히 설명할 수 있도록 돕는다.', 'isbn13': '9791158392239', 'cover': 'https://image.aladin.co.kr/product/25155/25/cover/k282633473_1.jpg', 'publisher': '위키북스', 'priceSales': 40500, 'pubDate': '2020-09-24', 'author': '이웅모 지음'}


In [92]:
from PIL import Image
from io import BytesIO


def get_images(image_url):
    res = requests.get(image_url)
    bytes_img = res.content
    img = Image.open(BytesIO(bytes_img))
    img.save(f"thumbnails.jpg")

In [93]:
get_images(extract_example1['cover'])

In [ ]:
# 와 마무리!